### Importing all the relevant libraries

In [4]:
import matplotlib
import matplotlib.pyplot as plt
import pkg_resources
import pandas as pd
import numpy as np
import sobol_seq
import types

### Calling the dataframe we'll be working on

In [ ]:
df = pd.read_excel('C:/Users/Samuele/Dropbox (Personal)/ECGC_Barcelona/1.Projects/01-DG_REGIO/Database_Final_UPD(3).xlsx')

### Generate the series of quasi-random numbers for the analysis

In [ ]:
quasiRandom_df = pd.DataFrame(sobol_seq.i4_sobol_generate(6,1000))
DistributionFiMax = 0.8+quasiRandom_df[0]*0.2
DistributionFiMin = 0.2+quasiRandom_df[1]*0.2

### Rearrange the dataframe for our convenience

In [ ]:
dfb = df[['ProgrammingPeriod','Country','NUTS1Code','NUTS2Code','Year','CF_TOTAL','EAGGF','ERDF_TOTAL','ESF']].copy()
df2 = dfb.melt(id_vars=['ProgrammingPeriod','Country','NUTS1Code','NUTS2Code','Year'],var_name='FundingScheme')
df3 = df2.pivot_table(index=['ProgrammingPeriod','FundingScheme','Country','NUTS1Code','NUTS2Code'],columns='Year', values='value')
df4 = df3.dropna(how='all').fillna(0)

### Excluding years before the commencing of the programming periods

In [ ]:
dummy = []

diff = int(np.sort(np.array(list(set(df.ProgrammingPeriod))))[0][5:])+6-\
int(np.sort(np.array(list(set(df.ProgrammingPeriod))))[0][:4])

d_Var = pd.Series([k/diff for k in range(1,diff+1)],[y for y in \
range(int(np.sort(np.array(list(set(df.ProgrammingPeriod))))[0][:4]),
int(np.sort(np.array(list(set(df.ProgrammingPeriod))))[0][5:])+6)])

dummy.append(d_Var)

for pp in np.sort(np.array(list(set(df.ProgrammingPeriod))))[1:]:
    diff = int(pp[5:])+4-int(pp[:4])
    
    d_Var = pd.Series([k/diff for k in range(1,diff+1)],[y for y in range(int(pp[:4]),int(pp[5:])+4)])
    
    dummy.append(d_Var)

### Normalise the cumulative figures

In [ ]:
df5 = df4.copy()

Norm_df6 = ((df5.cumsum(axis=1).T/df5.cumsum(axis=1).max(axis=1).values).T).dropna(how='all')

### Define the two outcome variables

In [ ]:
for ip, pp in enumerate(np.sort(np.array(list(set(df.ProgrammingPeriod))))):
    Norm_df6.loc[pd.IndexSlice[pp,:,:,:],'$\delta$'] = (Norm_df6.loc[pd.IndexSlice[pp,:,:,:],dummy[ip].index]-dummy[ip].T).cumsum(axis=1).iloc[:,-1]    

In [ ]:
df7 = Norm_df6.groupby(['ProgrammingPeriod','FundingScheme'])
ld = []
for idx, df7b in df7:
    df8 = df7b.copy()
    df8['$\mu$']=(df8['$\delta$'].max()-df8['$\delta$'])/(df8['$\delta$'].max()-df8['$\delta$'].min())
    ld.append(df8)
Norm_df7 = pd.concat(ld)

### Set the trigger for the number of years the residual expenditure gets spread onto on the last eligible year of the programming perido

In [ ]:
df7c = Norm_df7.groupby(['ProgrammingPeriod'])
ld1 = []
co = -1
for idx, df7d in df7c:
    co+=1
    for iq,qr in enumerate(quasiRandom_df[2]):
        df8b = df7d.copy()
        df8b[0]=(qr*(df7d['$\mu$']*(len(dummy[co])-1)).astype(int)).astype(int)+1
        for il in range(1,len(dummy[co])):
            df8b[il]=df8b[0]-il
        df8b[df8b<1]=1
        cd = [il0 for il0 in range(len(dummy[co]))]
        df8b['value']=iq
        cd.append('value')
        df8b = df8b[cd]
        ld1.append(df8b)
years = pd.concat(ld1)
years.set_index('value', append=True, inplace=True)

### Define the yearly residues

In [ ]:
def A(n):
    return [(2**j)/(2**n-1) for j in reversed(range(n))]

B9 = []
for k in reversed(range(2,11)):
    B9.append(pd.DataFrame([A(y) for y in range(1,k)],index=[y for y in range(1,k)],
                           columns=[y for y in range(1,k)]).fillna(0).sort_values(by=1,ascending=False))


In [ ]:
Payments = df5.loc[Norm_df6.index].copy()

### Evaluate the expenditure figures

In [ ]:
eg  = df5.loc[Norm_df6.index].groupby('ProgrammingPeriod')
co = -1

Expend = []
for r in range(len(quasiRandom_df)):
    co = -1
    Expe = []
    for ie, Eg in eg:
        co += 1
        Exp = Eg.copy()
        if ie == '1989-1993':
            y = [yr for yr in range(int(ie[5:])+5,df5.columns.max())]

            Exp[int(ie[5:])+5]=Eg[y].agg('sum',axis=1)

            Exp.loc[:,y[1:]] = 0

            Exp[int(ie[5:])+5] = Exp[int(ie[5:])+5]*(DistributionFiMax[r]- 
            Norm_df7.loc[pd.IndexSlice[ie,:,:,:],'$\mu$']*(DistributionFiMax[r]-DistributionFiMin[r]))

            for iy2,y2 in enumerate(range(int(ie[:4]),int(ie[5:])+5)):
                Exp[y2] = Eg[y2]*(DistributionFiMax[r]- 
                Norm_df7.loc[pd.IndexSlice[ie,:,:,:],'$\mu$']*(DistributionFiMax[r]-DistributionFiMin[r]))

                for iy3,y3 in enumerate(reversed(range(y2,int(ie[5:])+5))):
                    Exp[y2]+=Eg[y3+1]*(1-DistributionFiMax[r]+ 
                    Norm_df7.loc[pd.IndexSlice[ie,:,:,:],'$\mu$']*(DistributionFiMax[r]-DistributionFiMin[r]))*\
                    B9[iy3+len(dummy[0])-len(dummy[co])].loc[years.loc[pd.IndexSlice[ie,:,:,:,:,r],iy3].values,
                                                     (y3+1-y2)].values

            Exp[int(ie[:4])] += Eg[int(ie[:4])]*(1-DistributionFiMax[r]+
            Norm_df7.loc[pd.IndexSlice[ie,:,:,:,:],'$\mu$']*(DistributionFiMax[r]-DistributionFiMin[r]))

        elif ie == '1994-1999' or ie == '2000-2006':
            y = [yr for yr in range(int(ie[5:])+3,df5.columns.max())]

            Exp[int(ie[5:])+3]=Eg[y].agg('sum',axis=1)

            Exp.loc[:,y[1:]] = 0

            Exp[int(ie[5:])+3] = Exp[int(ie[5:])+3]*(DistributionFiMax[r]- 
            Norm_df7.loc[pd.IndexSlice[ie,:,:,:],'$\mu$']*(DistributionFiMax[r]-DistributionFiMin[r]))

            for iy2,y2 in enumerate(range(int(ie[:4]),int(ie[5:])+3)):
                Exp[y2] = Eg[y2]*(DistributionFiMax[r]- 
                Norm_df7.loc[pd.IndexSlice[ie,:,:,:],'$\mu$']*(DistributionFiMax[r]-DistributionFiMin[r]))

                for iy3,y3 in enumerate(reversed(range(y2,int(ie[5:])+3))):
                    Exp[y2]+=Eg[y3+1]*(1-DistributionFiMax[r]+ 
                    Norm_df7.loc[pd.IndexSlice[ie,:,:,:],'$\mu$']*(DistributionFiMax[r]-DistributionFiMin[r]))*\
                    B9[iy3+len(dummy[0])-len(dummy[co])].loc[years.loc[pd.IndexSlice[ie,:,:,:,:,r],iy3].values,
                                                     (y3+1-y2)].values

            Exp[int(ie[:4])] += Eg[int(ie[:4])]*(1-DistributionFiMax[r]+
            Norm_df7.loc[pd.IndexSlice[ie,:,:,:,:],'$\mu$']*(DistributionFiMax[r]-DistributionFiMin[r]))      

        else:
            Exp[df5.columns.max()]=Eg[df5.columns.max()]*(DistributionFiMax[r]- 
            Norm_df7.loc[pd.IndexSlice[ie,:,:,:],'$\mu$']*(DistributionFiMax[r]-DistributionFiMin[r]))

            for iy2,y2 in enumerate(reversed(range(int(ie[:4]),df5.columns.max()))):
                Exp[y2] = Eg[y2]*(DistributionFiMax[r]- 
                Norm_df7.loc[pd.IndexSlice[ie,:,:,:],'$\mu$']*(DistributionFiMax[r]-DistributionFiMin[r]))

                for iy3,y3 in enumerate(reversed(range(y2,df5.columns.max()))):
                    Exp[y2]+=Eg[y3+1]*(1-DistributionFiMax[r]+ 
                    Norm_df7.loc[pd.IndexSlice[ie,:,:,:],'$\mu$']*(DistributionFiMax[r]-DistributionFiMin[r]))*\
                    B9[iy3+len(dummy[0])-len(dummy[co])].loc[years.loc[pd.IndexSlice[ie,:,:,:,:,r],iy3].values,
                                                     (y3+1-y2)].values

            Exp[int(ie[:4])] += Eg[int(ie[:4])]*(1-DistributionFiMax[r]+
            Norm_df7.loc[pd.IndexSlice[ie,:,:,:,:],'$\mu$']*(DistributionFiMax[r]-DistributionFiMin[r]))

        Expe.append(Exp)
    Expen = pd.concat(Expe)
    Expen['r']=r
    Expend.append(Expen)
Expenditure = pd.concat(Expend)
Expenditure.set_index('r', append=True, inplace=True)

In [ ]:
Expenditure.to_csv('Expenditure.csv')

### One can finally extract all the relevant features for the analysis

In [366]:
Feature_dic = {'Mean':Expenditure.groupby(['ProgrammingPeriod','FundingScheme','Country','NUTS1Code','NUTS2Code']).mean(),
              'Std':Expenditure.groupby(['ProgrammingPeriod','FundingScheme','Country','NUTS1Code','NUTS2Code']).std()}

Feature_pivot_dic = Feature_dic.copy()

for fk in Feature_pivot_dic.keys():
    Feature_pivot_dic[fk] =  pd.pivot_table(Feature_dic[fk].stack().reset_index(),values = 0,columns = 'FundingScheme',
                                        index=['ProgrammingPeriod','Country','NUTS1Code','NUTS2Code','Year']).reset_index()

In [368]:
writer = pd.ExcelWriter('PivotedData.xlsx', engine='xlsxwriter')

for fk in Feature_pivot_dic.keys():
    Feature_pivot_dic[fk].to_excel(writer, sheet_name=fk)
    
writer.save()